#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [ ]:
class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size

class Node:
    def __init__(self, split_feature=None, split_val=None, child_node_a=None, child_node_b=None, val=None, depth=None):
        self.split_feature = split_feature 
        self.split_val = split_val 
        self.depth = depth 
        self.child_node_a = child_node_a 
        self.child_node_b = child_node_b 
        self.val = val 

    def predict(self, data, tree):
        return np.array([self.travers_tree(x, tree) for x in data]) #twórze listę, podaje x i buduje drzewo

    def travers_tree(self, x, tree): #metoda rekurencyjna, która będzie przewidywać na podstawie drzewa
        if tree.is_leaf_node(): #jeśli to węzeł końcowy, zwracamy wartość
            return tree.val
            
        if x[tree.split_feature] <= tree.split_val: #jeśli nie końcowy, patrzymy gdzie iść w lewo lub w prawo
            return self.travers_tree(x, tree.child_node_a) #wlewo
        return self.travers_tree(x, tree.child_node_b) #wprawo

    def is_leaf_node(self): #metoda sprawdzania na węzł końcowy
        return self.val is not None

class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0 #glebokosc
        self.max_depth = max_depth #max glebokosc
        self.min_samples = 10 # min ilosc samples
        self.tree = None #dzewo

    def fit(self, x_train, y_train): #metoda nauki, budowania drzewa
        self.tree = self.build_tree(x_train, y_train) #wywołana metoda build_tree, aby zbudować drzewo

    def entropy(self, y): #entropia
        hist = np.bincount(y) #definicja częstotliwości klas
        ps = hist / len(y) # obliczanie udziału każdej klasy
        return -np.sum([p * np.log2(p) for p in ps if p > 0]) # obliczenie entropii

    def get_information_gain(self, parent_group, child_group_a, child_group_b): # Obliczanie information gain
        if len(np.unique(child_group_a)) == 1: #jeśli w węźle zostało tylko jedno miejsce, zatrzymujemy się wtedy zawartość informacji wynosi 0
            return 0
        
        n = len(child_group_a) #określanie liczbę obserwacji w węźle nadrzędnym
        parent = self.entropy(y) #entropia perents wezla
       
        #określienie wskaźnikow obserwacji, które przeszły na lewą i prawa stronę. 
        #flatten() zamienia kolumnę na str
        left_indexes = np.argwhere(parent_group <= child_group_b).flatten() 
        right_indexes = np.argwhere(parent_group > child_group_b).flatten() 
        
        #obliczanie entropii po podzieleniu w lewym i prawym węźle
        e_l, n_l = self.entropy(child_group_a[left_indexes]), len(left_indexes) 
        e_r, n_r = self.entropy(child_group_a[right_indexes]), len(right_indexes) 
        
        #entropia po podzieleniu 
        child = (n_l / n) * e_l + (n_r / n) * e_r 
        return parent - child #zwracam information_gain

    def get_best_feature_split(self, feature_values, classes): #metoda efektywnej separacji features
        labels = np.unique(classes) 
        count = [list(classes).count(i) for i in labels] #liczenie każdej labels
        return labels[np.argmax(count)] #zwracam najbardziej czesty labels

    def get_best_split(self, data, classes): #metoda najefektywnej separacji 
        best_feature, best_threshold = None, None #najlepszy feature and threshold
        best_gain = -1              
        
        #algorytm prochodzienia po wszystkiem cecham 
        for i in range(data.shape[1]): 
            thresholds = np.unique(data[:, i]) 
            for threshold in thresholds: 
                gain = self.get_information_gain(data[:, i], classes, threshold)
                if gain > best_gain: 
                    best_gain = gain 
                    best_feature = i 
                    best_threshold = threshold 
        return best_feature, best_threshold 

    def build_tree(self, data, classes, depth=0): # rekurencyjne budowanie drzewa
        n_samples, n_features = data.shape 
        n_labels = len(np.unique(classes)) #sprawdzanie, ile labels zostało, aby uniknąć sytuacji, w której została jedna label
        
        if n_samples <= self.min_samples or depth >= self.max_depth or n_labels == 1: ## jeśli n_samples to minimalna dozwolona wartość lub głębokość to maksymalna dozwolona wartość lub n_labels to pozostała jedna klasa
            return Node(val=self.get_best_feature_split(0, classes)) 
        
        best_feature, best_threshold = self.get_best_split(data, classes) 
        
        #określa, które podziały poszły w lewo, a które w prawo, w tym celu określamy lewe lub prawe indeksy.
        left_indexes = np.argwhere(data[:, best_feature] <= best_threshold).flatten() 
        right_indexes = np.argwhere(data[:, best_feature] > best_threshold).flatten() 
        
        if len(left_indexes) == 0 or len(right_indexes) == 0: #jeśli lewe lub prawe indeksy są puste
            return Node(val=self.get_best_feature_split(0, classes)) #zwracamy w węźle wartość, którą wcześniej obliczyliśmy metodą get_best_feature_split
        
        #budujemy drzewo 
        left = self.build_tree(data[left_indexes, :], classes[left_indexes], depth + 1) 
        right = self.build_tree(data[right_indexes, :], classes[right_indexes], depth + 1) 
        
        return Node(best_feature, best_threshold, left, right) 

    def predict(self, data): # metoda przewidywania wytrenowanej sieci danych
        return self.tree.predict(data, self.tree) 


In [ ]:
dc = DecisionTreeClassifier(3)
dc.fit(x_train, y_train)

prediction = dc.predict(x_test)
print(prediction)
#print(x_train)
np.sum(prediction == y_test) / len(y_test)

[2 2 2 1 0 2 1 0 0 1 2 0 1 2 2]


0.9333333333333333

### Przy ustawieniu test_size=0.1, random_state=123:

Dżewo [2 2 2 1 0 2 1 0 0 1 2 0 1 2 2]

Accuracy 0.9333333333333333


---


### Przy ustawieniu test_size=0.2:
Accuracy 0.9666666666666667

Accuracy 1.0

Accuracy 0.8333333333333334

---
### Przy ustawieniu test_size=0.3:

Accuracy 0.9777777777777777

Accuracy 0.8444444444444444

Accuracy 0.9333333333333333


